In [0]:
pip install azure-storage-blob

Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-8d030fca-e480-4730-8256-08f05207c02a
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install wordcloud

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
!pip install pandas_bokeh
import pandas_bokeh


     |███████

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 7.8 MB 33.1 MB/s 
     |████████████████████████████████| 300 kB 58.5 MB/s 
     |████████████████████████████████| 738 kB 51.7 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-8d030fca-e480-4730-8256-08f05207c02a/bin/python -m pip install --upgrade pip' command.


In [0]:
from ipywidgets import interact
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [0]:
import requests
import pandas as pd
import json
import plotly
import plotly.graph_objects as go
import gzip
import matplotlib.pyplot as plt
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.core.pipeline.transport import HttpResponse
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, explode,to_date, date_format, count, desc, countDistinct, regexp_extract,regexp_replace, dayofweek,when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from wordcloud import WordCloud
from nltk.corpus import stopwords
from pyspark.sql.functions import dayofmonth, countDistinct, to_date, hour
from pyspark.sql.functions import explode, lower, length, trim, regexp_replace

In [0]:
spark = SparkSession.builder.getOrCreate()

storage_account_name = "pod4projectstorage"
storage_account_access_key = "2hClDrVPLGX4QBDBk8OylAkHqczIQfDja66Yl488rmj/0+vb+CAzOxL5qMe5XyM9ZupgwveVRm3N+AStriO5vg=="

container_name = "data"
directory_path = ""

spark.conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)

df = spark.read.parquet("wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net/" + directory_path)


In [0]:
push_df = df.filter(df.type == "PushEvent")

In [0]:
for col_name in push_df.columns:
    new_col_name = col_name.replace("payload.", "")
    push_df = push_df.withColumnRenamed(col_name, new_col_name)
push_df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|     type|public|          created_at|                 org|               actor|                repo|action|              before|             commits|repository_id|review|pages|    push_id|pusher_type|                 ref|size|ref_type|
+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|PushEvent|  true|2023-06-01T00:00:00Z|{https://avatars....|{https://avatars....|{474293004, kuda-...|  null|6f6f6712bea287e78...|[{{robbycaesar@gm...|    474293004|  null| null|13828565359|       null|  refs/heads/develop|   2|    null|
|PushEvent|  true|2023-06-01T00:00:00Z|         

In [0]:
push_df = push_df.withColumn("commit", explode("commits")).withColumn("commit_author", col("commit.author.name")).withColumn("commit_message", col("commit.message")).withColumn("login", col("actor.login")).withColumn("url", col("commit.url")).withColumn("day_of_week", date_format(to_date(col("created_at"), "yyyy-MM-dd'T'HH:mm:ss'Z'"), "EEEE")).withColumn("domain", regexp_extract(col("url"), r".*repos/(.*)/commits/.*", 1)).drop("commits", "commit","url")

In [0]:
count_by_day = push_df.groupBy("day_of_week").count()
count_by_day.show()

+-----------+--------+
|day_of_week|   count|
+-----------+--------+
|    Tuesday|14037309|
|  Wednesday|14017805|
|   Thursday|17147687|
|     Monday|13401349|
|     Friday|16905591|
|   Saturday|10488101|
|     Sunday|10524022|
+-----------+--------+



In [0]:
count_pd = count_by_day.toPandas()
fig1 = go.Figure(data=go.Scatter(x=count_pd["day_of_week"], y=count_pd["count"], mode='lines+markers', line=dict(color='blue')))
fig1.update_traces(marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6, line_width=2)
fig1.update_layout(title="Number of Commits by Day of the Week", xaxis_title="Day of the Week", yaxis_title="Count", width=1000, height=550)
fig1.update_xaxes(showgrid=False)
fig1.update_yaxes(showgrid=False, dtick=1000000)

fig1.show()

In [0]:
commit_msg_by_author_counts = push_df.groupBy("commit_message").agg(countDistinct(col("commit_author")).alias("author_count")).orderBy(desc("author_count"))

In [0]:
commit_msg_by_author_counts.show()

+--------------------+------------+
|      commit_message|author_count|
+--------------------+------------+
|    Update README.md|      468224|
|Add files via upload|      386051|
|    Create README.md|      137749|
|   Update index.html|       77138|
|              update|       65047|
|        first commit|       35443|
|                test|       35039|
|        Create CNAME|       31328|
|                 fix|       29033|
|      Initial commit|       25405|
|        Delete CNAME|       23411|
|             Updates|       23084|
|    Delete README.md|       23005|
|      initial commit|       21114|
|                   .|       20194|
|              commit|       18142|
|              Update|       16731|
|           README.md|       15934|
|   Delete index.html|       15845|
|   Create index.html|       14913|
+--------------------+------------+
only showing top 20 rows



In [0]:
top_commit_messages = commit_msg_by_author_counts.limit(10).toPandas()
fig2 = go.Figure(data=go.Bar(x=top_commit_messages['commit_message'], y=top_commit_messages['author_count'], marker =dict(color='blue')))
fig2.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.7)
fig2.update_layout(title='Top 10 Commit Messages by Author Count', xaxis_title='Commit Message', yaxis_title='Author Count',width = 1000, height=550)
fig2.show()

In [0]:
commit_by_domain_counts = push_df.groupBy("domain").agg(count("*").alias("commit_count")).orderBy(desc("commit_count"))

In [0]:
commit_by_domain_counts.show()

+--------------------+------------+
|              domain|commit_count|
+--------------------+------------+
|  happyfish2024/mins|      366026|
|mizzled/comitigation|      328125|
|   rmayr/rediffusion|      318511|
|mhutchinson-witne...|      296457|
|rmayr/mhutchinson...|      270982|
|WolseyBankWitness...|      258370|
|mhutchinson-witne...|      256785|
|WolseyBankWitness...|      247628|
|networkoperator/d...|      226237|
|bullet-dev-team/p...|      218726|
|       cmd777/commit|      199996|
|DF-Master/DF-Mast...|      154060|
|bullet-dev-team/d...|      115191|
|   hotspotlab/hourly|       97917|
|    Zebrainy/upptime|       95046|
|    vpnsuperapp/fast|       94775|
|  predictcrypto/pins|       93787|
|Roblox-Thot/throw...|       88420|
|splashingdiamonds...|       83020|
|       zomeru/zomeru|       80613|
+--------------------+------------+
only showing top 20 rows



In [0]:
top_commit_domains = commit_by_domain_counts.limit(10).toPandas()
fig3 = go.Figure(data=go.Bar(x=top_commit_domains['domain'], y=top_commit_domains['commit_count'],marker=dict(color='blue')))
fig3.update_traces( marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6,width=0.7)
fig3.update_layout(title='Top 10 Commit domains', xaxis_title='domains', yaxis_title='Count', width = 1000, height=550)
fig3.show()

In [0]:
issues_df = df.filter(df.type=='PushEvent')

In [0]:
for col_name in issues_df.columns:
    new_col_name = col_name.replace("payload.", "")
    issues_df = issues_df.withColumnRenamed(col_name, new_col_name)
issues_df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|     type|public|          created_at|                 org|               actor|                repo|action|              before|             commits|repository_id|review|pages|    push_id|pusher_type|                 ref|size|ref_type|
+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|PushEvent|  true|2023-06-01T00:00:00Z|{https://avatars....|{https://avatars....|{474293004, kuda-...|  null|6f6f6712bea287e78...|[{{robbycaesar@gm...|    474293004|  null| null|13828565359|       null|  refs/heads/develop|   2|    null|
|PushEvent|  true|2023-06-01T00:00:00Z|         

In [0]:
descrip = issues_df.select(explode('commits.message').alias("message"))

In [0]:
grouped_df = df.groupBy("type").count().orderBy("count", ascending=False)

In [0]:
pd_data = grouped_df.toPandas()
pd_data

,type,count
0,PushEvent,60183436


In [0]:
fig4 = go.Figure([go.Bar(x=pd_data['type'], y=pd_data['count'],marker=dict(color='orangered'))])
fig4.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig4.update_layout(title_text='Event Counts',xaxis=dict(tickangle=90), width =1000, height=550)
fig4.show()

In [0]:
push_df = issues_df

In [0]:
for col_name in df.columns:
    new_col_name = col_name.replace("payload.", "")
    push_df = push_df.withColumnRenamed(col_name, new_col_name)
push_df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|     type|public|          created_at|                 org|               actor|                repo|action|              before|             commits|repository_id|review|pages|    push_id|pusher_type|                 ref|size|ref_type|
+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|PushEvent|  true|2023-06-01T00:00:00Z|{https://avatars....|{https://avatars....|{474293004, kuda-...|  null|6f6f6712bea287e78...|[{{robbycaesar@gm...|    474293004|  null| null|13828565359|       null|  refs/heads/develop|   2|    null|
|PushEvent|  true|2023-06-01T00:00:00Z|         

In [0]:
hour_df = push_df.withColumn('hour', hour(push_df['created_at']))

In [0]:
hour_df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+-------------------+--------------------+-------------------+-----------+--------------------+----+
|     type|public|          created_at|                 org|               actor|                repo|action|              before|repository_id|review|pages|    push_id|pusher_type|                 ref|size|ref_type|      commit_author|      commit_message|              login|day_of_week|              domain|hour|
+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+-------------------+--------------------+-------------------+-----------+--------------------+----+
|PushEvent|  true|2023-06-01T00:00:00Z|{https://avat

In [0]:
grouped_hour = hour_df.groupBy("hour").count().orderBy("count", ascending=False)

In [0]:
pd_hour = grouped_hour.toPandas().sort_values(by='hour')

In [0]:
pd_hour['hour'] = pd_hour['hour'] + 1

In [0]:
fig5 = go.Figure([go.Scatter(x=pd_hour['hour'], y=pd_hour['count'], line=dict(color='blue', width=3))])
fig5.update_traces(marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)
fig5.update_layout(title_text='Commits by Hour', width=1000, height=550)
fig5.update_xaxes(showgrid=False, dtick=1)
fig5.update_yaxes(showgrid=False)
fig5.show()

In [0]:
result = push_df.groupBy(dayofmonth("created_at").alias("Day")).agg(countDistinct("actor.id").alias("Users")).orderBy("Day")

In [0]:
result.show()

+---+------+
|Day| Users|
+---+------+
|  1|403493|
|  2|382942|
|  3|277450|
|  4|288128|
|  5|396094|
|  6|406564|
|  7|402209|
|  8|395617|
|  9|377544|
| 10|275837|
| 11|284741|
| 12|394237|
| 13|409951|
| 14|412486|
| 15|398098|
| 16|378829|
| 17|273853|
| 18|276608|
| 19|378492|
| 20|407018|
+---+------+
only showing top 20 rows



In [0]:
pd_date = result.toPandas()

In [0]:
fig6 = go.Figure([go.Scatter(x=pd_date['Day'], y=pd_date['Users'], line=dict(color='blue', width=3))])
max_users = pd_date['Users'].max()
fig6.update_traces(marker=dict(color=['yellowgreen' if val == max_users else 'blue' for val in pd_date['Users']]),
                    marker_line_color='rgb(8, 48, 107)', marker_line_width=1.5, opacity=0.6)
fig6.update_layout(title_text='Commits by Day of the Month', width=1000, height=550)
fig6.update_xaxes(showgrid=False)
fig6.update_yaxes(showgrid=False)
fig6.show()

In [0]:
fig6 = go.Figure([go.Bar(x=pd_date['Day'], y=pd_date['Users'], marker=dict(
        color=['yellowgreen' if val == pd_date['Users'].max() else 'orangered' for val in pd_date['Users']]))])
fig6.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.7)
fig6.update_layout(title_text='Commits by Day of the Month',width = 1000, height = 550)
fig6.show()

In [0]:
grouped_pushdf = push_df.select('org.login').dropna()

In [0]:
grouped_pushdf = grouped_pushdf.groupBy("login").count().orderBy("count", ascending=False)

In [0]:
grouped_pushdf.show()

+--------------------+------+
|               login| count|
+--------------------+------+
|     networkoperator|226237|
|          tf-vcs-e2e|133750|
| redhat-appstudio-qe|121568|
|               hmcts|101831|
|            Zebrainy| 95049|
|           newstools| 77204|
|  MetadataGitTesting| 75429|
|           hashicorp| 69597|
|        magicwave-io| 66480|
|            Risk-DAO| 65764|
|           microsoft| 65709|
|         isomerpages| 64544|
|             pytorch| 60097|
|       multipath-tcp| 56050|
|          1Panel-dev| 55996|
|            odoo-dev| 55403|
|   ministryofjustice| 53749|
|Ultimate-Hosts-Bl...| 50515|
|             demisto| 50294|
|linux-mailinglist...| 49663|
+--------------------+------+
only showing top 20 rows



In [0]:
pd_pushdf = grouped_pushdf.toPandas()
pd_pushdf = pd_pushdf.head()

In [0]:
pd_pushdf

,login,count
0,networkoperator,226237
1,tf-vcs-e2e,133750
2,redhat-appstudio-qe,121568
3,hmcts,101831
4,Zebrainy,95049


In [0]:
fig7 = go.Figure([go.Bar(x=pd_pushdf['login'], y=pd_pushdf['count'],marker=dict(color='blue'))])
fig7.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.5)
fig7.update_layout(title_text='Organizations with highest number of commits',width = 1000, height = 550)
fig7.show()

In [0]:
exploded_msg = push_df.select(explode('commits.message').alias("message"))

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-864829005233402>:1
----> 1 exploded_msg = push_df.select(explode('commits.message').alias("message"))

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3023, in DataFrame.select(self, *cols)
   2978 def select(self, *cols: "ColumnOrName") -> "DataFrame":  # type: ignore[misc]
   2979     """Projects a set of expressions and returns a new :class:`DataFrame`.
   2980 
   2981     .. versionadded:: 1.3.0
   (...)
   3021     +-----+---+
   3022     """
-

In [0]:
exploded_msg.dropna()
exploded_msg.show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-864829005233403>:1
----> 1 exploded_msg.dropna()
      2 exploded_msg.show()

NameError: name 'exploded_msg' is not defined

In [0]:
pandas_bokeh.plot_grid([[fig1.show(), fig2.show()], [fig3.show(), fig4.show()], [fig5.show(), fig6.show(), fig7.show()]], width=200, height=200)
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [0]:
exploded_msg = exploded_msg.withColumn("message", trim(lower(regexp_replace(exploded_msg.message, "[^a-zA-Z\\s]", ""))))

In [0]:
exploded_msg = exploded_msg.filter(length(exploded_msg.message) > 0)


In [0]:
exploded_msg.show()

+--------------------+
|             message|
+--------------------+
|latest data mon j...|
|ajustando margene...|
|svgs para la pagi...|
|merge pull reques...|
|automated update ...|
|automated update jun|
|update dockerimag...|
|silver i title  t...|
|updating online s...|
|         print array|
|            homebrew|
|          frustrated|
|first advanced co...|
|create apresentao...|
|update utd court ...|
|              devops|
|     update readmemd|
|updated fmdataapi...|
|merge remotetrack...|
|authenticationsto...|
+--------------------+
only showing top 20 rows



In [0]:
tokenizer = Tokenizer(inputCol="message", outputCol="tokens")
tz_msg = tokenizer.transform(exploded_msg)

In [0]:
tz_msg.show()

+--------------------+--------------------+
|             message|              tokens|
+--------------------+--------------------+
|latest data mon j...|[latest, data, mo...|
|ajustando margene...|[ajustando, marge...|
|svgs para la pagi...|[svgs, para, la, ...|
|merge pull reques...|[merge, pull, req...|
|automated update ...|[automated, updat...|
|automated update jun|[automated, updat...|
|update dockerimag...|[update, dockerim...|
|silver i title  t...|[silver, i, title...|
|updating online s...|[updating, online...|
|         print array|      [print, array]|
|            homebrew|          [homebrew]|
|          frustrated|        [frustrated]|
|first advanced co...|[first, advanced,...|
|create apresentao...|[create, apresent...|
|update utd court ...|[update, utd, cou...|
|              devops|            [devops]|
|     update readmemd|  [update, readmemd]|
|updated fmdataapi...|[updated, fmdataa...|
|merge remotetrack...|[merge, remotetra...|
|authenticationsto...|[authentic

In [0]:
remover = StopWordsRemover(inputCol='tokens', outputCol='fil_msg')
tz_msg = remover.transform(tz_msg).drop('tokens')

In [0]:
tz_msg.show()

+--------------------+--------------------+
|             message|             fil_msg|
+--------------------+--------------------+
|latest data mon j...|[latest, data, mo...|
|ajustando margene...|[ajustando, marge...|
|svgs para la pagi...|[svgs, para, la, ...|
|merge pull reques...|[merge, pull, req...|
|automated update ...|[automated, updat...|
|automated update jun|[automated, updat...|
|update dockerimag...|[update, dockerim...|
|silver i title  t...|[silver, title, ,...|
|updating online s...|[updating, online...|
|         print array|      [print, array]|
|            homebrew|          [homebrew]|
|          frustrated|        [frustrated]|
|first advanced co...|[first, advanced,...|
|create apresentao...|[create, apresent...|
|update utd court ...|[update, utd, cou...|
|              devops|            [devops]|
|     update readmemd|  [update, readmemd]|
|updated fmdataapi...|[updated, fmdataa...|
|merge remotetrack...|[merge, remotetra...|
|authenticationsto...|[authentic